<a href="https://colab.research.google.com/github/T-Vinita/Insights-on-Amazon-Sales-Report-/blob/main/insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv("Amazon Sale Report.csv")  # Change filename as needed

print(df.head())

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df = df.dropna(subset=['Category', 'Amount', 'Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.day_name()

# --- Sales Overview ---
sales_summary = df.groupby(['Year', 'Month'])['Amount'].sum().reset_index()
sales_summary = sales_summary.sort_values(['Year', 'Month'])

# --- Product Analysis ---
product_sales = df.groupby('Category')['Amount'].sum().sort_values(ascending=False).reset_index()
# The 'Quantity' column is missing from the dataset, so we cannot calculate product quantity.
# product_qty = df.groupby('Category')['Quantity'].sum().sort_values(ascending=False).reset_index()


# --- Fulfillment Analysis ---
fulfillment_counts = df['Fulfilment'].value_counts().reset_index()
fulfillment_counts.columns = ['Fulfillment Method', 'Count']

# --- Geographical Analysis ---
geo_sales = df.groupby('ship-state')['Amount'].sum().sort_values(ascending=False).reset_index()

# --- Customer Segmentation (if Customer ID or Address available) ---
# If there is no customer ID, you can segment by state or high/low spenders
df['High Value'] = df['Amount'] > df['Amount'].median()

# --- Save to CSV for Power BI ---
df.to_csv('cleaned_amazon_sales.csv', index=False)
sales_summary.to_csv('sales_summary.csv', index=False)
product_sales.to_csv('product_sales.csv', index=False)
# We cannot save 'product_qty.csv' because the 'Quantity' column is missing.
# product_qty.to_csv('product_qty.csv', index=False)
fulfillment_counts.to_csv('fulfillment_counts.csv', index=False)
geo_sales.to_csv('geo_sales.csv', index=False)

print("✅ Files saved. Ready to import into Power BI.")

# --- Optional: Basic Plots in Python (just for initial checks) ---
plt.figure(figsize=(10, 5))
sns.lineplot(data=sales_summary, x='Month', y='Amount', hue='Year', marker="o")
plt.title("Monthly Sales Trend")
plt.show()

plt.figure(figsize=(10, 5))
sns.barplot(data=product_sales, x='Amount', y='Category')
plt.title("Sales by Product Category")
plt.show()

plt.figure(figsize=(6, 6))
plt.pie(fulfillment_counts['Count'], labels=fulfillment_counts['Fulfillment Method'], autopct='%1.1f%%')
plt.title("Fulfillment Method Distribution")
plt.show()